In [2]:
import pandas as pd
import numpy as np

In [3]:
# Step 1: Read in the first 200K lines, with 8 columns
df = pd.read_csv('C:\\Users\\ajaco\\Desktop\\repos\\noreallyimfine\\ebird-project\\data\\ebd_relJan-2020.txt', sep='\t', nrows=2000000, usecols=['COMMON NAME', 'COUNTRY', 'STATE', 'COUNTY', 'LATITUDE', 'LONGITUDE', 'OBSERVATION DATE', 'OBSERVATION COUNT'])

print(df.shape)

df.head()

(2000000, 8)


,COMMON NAME,OBSERVATION COUNT,COUNTRY,STATE,COUNTY,LATITUDE,LONGITUDE,OBSERVATION DATE
0,Magnolia Warbler,2,United States,Illinois,Cook,41.775629,-87.583273,1995-08-27
1,White-rumped Sandpiper,4,Canada,Quebec,Manicouagan,49.216670,-68.150000,1993-11-07
2,Common Scoter,1,Sweden,Hallands län [SE-13],NaN,57.065084,12.243579,1998-02-21
3,Ring-billed Gull,15,Canada,Manitoba,South Interlake,50.193256,-97.137935,1985-04-14
4,Red-winged Blackbird,500,Canada,Manitoba,South Interlake,50.193256,-97.137935,1986-09-01


In [4]:
df.isnull().sum()

COMMON NAME               0
OBSERVATION COUNT         0
COUNTRY                   0
STATE                     0
COUNTY               292946
LATITUDE                  0
LONGITUDE                 0
OBSERVATION DATE          0
dtype: int64

In [5]:
# Rename columns for ease of use
df.rename(columns={
    'COMMON NAME': 'name',
    'OBSERVATION COUNT': 'observ_count',
    'COUNTRY': 'country',
    'STATE': 'state',
    'COUNTY': 'county',
    'LATITUDE': 'latitude',
    'LONGITUDE': 'longitude',
    'OBSERVATION DATE': 'observ_date'
}, inplace=True)

In [6]:
# Filter for just US birds
us_birds = df.query("country == 'United States'")

print(us_birds.shape)
us_birds.head()

(1095180, 8)


,name,observ_count,country,state,county,latitude,longitude,observ_date
0,Magnolia Warbler,2,United States,Illinois,Cook,41.775629,-87.583273,1995-08-27
6,Greater Yellowlegs,X,United States,Texas,Aransas,28.240392,-96.818819,1986-04-06
12,White-crowned Sparrow,X,United States,Arizona,Cochise,31.898164,-109.115932,1998-11-27
13,Green-winged Teal,11,United States,Idaho,Ada,43.609793,-116.206427,1982-12-18
14,Yellow-rumped Warbler,5,United States,Idaho,Ada,43.609793,-116.206427,1982-12-18


In [7]:
us_birds = us_birds.copy()

# Drop missing column values
us_birds.dropna(subset=['county'], inplace=True)

us_birds.isnull().sum()

name            0
observ_count    0
country         0
state           0
county          0
latitude        0
longitude       0
observ_date     0
dtype: int64

In [8]:
us_birds['bad_name'] = us_birds['name'].apply(lambda x: 0 if ("sp." in x) or ("(" in x) or ("/" in x) else 1)
us_birds.bad_name.value_counts()

1    1084942
0       8618
Name: bad_name, dtype: int64

In [9]:
mask = us_birds['bad_name'] == 0
us_birds = us_birds[~mask]
us_birds.shape

(1084942, 9)

In [10]:
# Replace 'X' in 'observ_count' with 1
us_birds['observ_count'] = us_birds['observ_count'].apply(lambda x: 1 if x == 'X' else x)

us_birds.head()

,name,observ_count,country,state,county,latitude,longitude,observ_date,bad_name
0,Magnolia Warbler,2,United States,Illinois,Cook,41.775629,-87.583273,1995-08-27,1
6,Greater Yellowlegs,1,United States,Texas,Aransas,28.240392,-96.818819,1986-04-06,1
12,White-crowned Sparrow,1,United States,Arizona,Cochise,31.898164,-109.115932,1998-11-27,1
13,Green-winged Teal,11,United States,Idaho,Ada,43.609793,-116.206427,1982-12-18,1
14,Yellow-rumped Warbler,5,United States,Idaho,Ada,43.609793,-116.206427,1982-12-18,1


In [11]:
# Convert 'observ_date' to datetime and extract year and month
us_birds.observ_date = pd.to_datetime(us_birds['observ_date'], infer_datetime_format=True)

us_birds['month'] = us_birds.observ_date.dt.month

us_birds.shape

(1084942, 10)

In [12]:
# 'season' column from month values
def month_to_season(x):
    if x in [12, 1, 2]:
        return 'Winter'
    elif x in [3, 4, 5]:
        return 'Spring'
    elif x in [6, 7, 8]:
        return 'Summer'
    else:
        return 'Fall'

us_birds['season'] = us_birds['month'].apply(month_to_season)
us_birds.head()

,name,observ_count,country,state,county,latitude,longitude,observ_date,bad_name,month,season
0,Magnolia Warbler,2,United States,Illinois,Cook,41.775629,-87.583273,1995-08-27,1,8,Summer
6,Greater Yellowlegs,1,United States,Texas,Aransas,28.240392,-96.818819,1986-04-06,1,4,Spring
12,White-crowned Sparrow,1,United States,Arizona,Cochise,31.898164,-109.115932,1998-11-27,1,11,Fall
13,Green-winged Teal,11,United States,Idaho,Ada,43.609793,-116.206427,1982-12-18,1,12,Winter
14,Yellow-rumped Warbler,5,United States,Idaho,Ada,43.609793,-116.206427,1982-12-18,1,12,Winter


In [13]:
us_birds.season.value_counts()['Spring']

415905

In [14]:
# Create merge column
us_birds['county_state'] = us_birds['county'] + us_birds['state']

us_birds.head()

,name,observ_count,country,state,county,latitude,longitude,observ_date,bad_name,month,season,county_state
0,Magnolia Warbler,2,United States,Illinois,Cook,41.775629,-87.583273,1995-08-27,1,8,Summer,CookIllinois
6,Greater Yellowlegs,1,United States,Texas,Aransas,28.240392,-96.818819,1986-04-06,1,4,Spring,AransasTexas
12,White-crowned Sparrow,1,United States,Arizona,Cochise,31.898164,-109.115932,1998-11-27,1,11,Fall,CochiseArizona
13,Green-winged Teal,11,United States,Idaho,Ada,43.609793,-116.206427,1982-12-18,1,12,Winter,AdaIdaho
14,Yellow-rumped Warbler,5,United States,Idaho,Ada,43.609793,-116.206427,1982-12-18,1,12,Winter,AdaIdaho


In [15]:
# Read in regions df
regions = pd.read_excel("C:\\Users\\ajaco\\Desktop\\repos\\noreallyimfine\\ebird-project\data\\URAmericaMapCountyList.xlsx", skiprows=3, usecols=['State', 'CountyName', 'RegionName'])

print(regions.shape)

regions.head()

(3142, 3)


,State,CountyName,RegionName
0,Alabama,"Autauga County, Alabama",01 Deep South
1,Alabama,"Baldwin County, Alabama",03 Gulf Coast
2,Alabama,"Barbour County, Alabama",01 Deep South
3,Alabama,"Bibb County, Alabama",02 Appohzarka
4,Alabama,"Blount County, Alabama",02 Appohzarka


In [16]:
# Strip leading whitespace
regions.State = regions.State.str.strip()

regions.State.unique()

array(['Alabama', 'Alaska', 'Arizona', 'Arkansas', 'California',
       'Colorado', 'Connecticut', 'Delaware', 'District of Columbia',
       'Florida', 'Georgia', 'Hawaii', 'Idaho', 'Illinois', 'Indiana',
       'Iowa', 'Kansas', 'Kentucky', 'Louisiana', 'Maine', 'Maryland',
       'Massachusetts', 'Michigan', 'Minnesota', 'Mississippi',
       'Missouri', 'Montana', 'Nebraska', 'Nevada', 'New Hampshire',
       'New Jersey', 'New Mexico', 'New York', 'North Carolina',
       'North Dakota', 'Ohio', 'Oklahoma', 'Oregon', 'Pennsylvania',
       'Rhode Island', 'South Carolina', 'South Dakota', 'Tennessee',
       'Texas', 'Utah', 'Vermont', 'Virginia', 'Washington',
       'West Virginia', 'Wisconsin', 'Wyoming'], dtype=object)

In [17]:
# Split region numbers off RegionName
regions['RegionName'] = regions['RegionName'].apply(lambda x: ' '.join(x.split()[1:]))

In [18]:
# Split state name off county
regions['CountyName'] = regions['CountyName'].apply(lambda x: x.split(',')[0])

regions.head()

,State,CountyName,RegionName
0,Alabama,Autauga County,Deep South
1,Alabama,Baldwin County,Gulf Coast
2,Alabama,Barbour County,Deep South
3,Alabama,Bibb County,Appohzarka
4,Alabama,Blount County,Appohzarka


In [19]:
# Change Alaska county names to match birds counties
county_dict = {
    'Aleutians East Borough': 'Aleutians East',
    'Aleutians West Census Area': 'Aleutians West',
    'Anchorage Municipality': 'Anchorage',
    'Bethel Census Area': 'Bethel',
    'Bristol Bay Borough': 'Bristol Bay',
    'Denali Borough': 'Denali',
    'Dillingham Census Area': 'Dillingham',
    'Fairbanks North Star Borough': 'Fairbanks North Star',
    'Haines Borough': 'Haines',
    'Hoonah-Angoon Census Area': 'Skagway-Hoonah-Angoon',
    'Juneau City and Borough': 'Juneau',
    'Kenai Peninsula Borough': 'Kenai Peninsula',
    'Ketchikan Gateway Borough': 'Ketchikan Gateway',
    'Kodiak Island Borough': 'Kodiak Island',
    'Kusilvak Census Area': 'Kusilvak',
    'Lake and Peninsula Borough': 'Lake and Peninsula',
    'Matanuska-Susitna Borough': 'Matanuska-Susitna',
    'Nome Census Area': 'Nome',
    'North Slope Borough': 'North Slope',
    'Northwest Arctic Borough': 'Northwest Arctic',
    'Petersburg Borough': 'Petersburg Borough',
    'Prince of Wales-Hyder Census Area': 'Prince of Wales-Outer Ketchikan',
    'Sitka City and Borough': 'Sitka',
    'Southeast Fairbanks Census Area': 'Southeast Fairbanks',
    'Valdez-Cordova Census Area': 'Valdez-Cordova',
    'Yakutat City and Borough': 'Yakutat',
    'Yukon-Koyukuk Census Area': 'Yukon-Koyukuk'
}

regions['CountyName'] = regions['CountyName'].apply(lambda x: county_dict[x] if x in county_dict.keys() else x)
regions.CountyName.nunique()

1877

In [20]:
# Remove 'Parish' from CountyName
regions['CountyName'] = regions['CountyName'].apply(lambda x: x if 'Parish' not in x else ' '.join(x.split()[:-1]))
regions.query("State == 'Louisiana'")

,State,CountyName,RegionName
1113,Louisiana,Acadia,Gulf Coast
1114,Louisiana,Allen,Deep South
1115,Louisiana,Ascension,Gulf Coast
1116,Louisiana,Assumption,Gulf Coast
1117,Louisiana,Avoyelles,Gulf Coast
...,...,...,...
1172,Louisiana,Webster,Deep South
1173,Louisiana,West Baton Rouge,Gulf Coast
1174,Louisiana,West Carroll,Deep South
1175,Louisiana,West Feliciana,Gulf Coast


In [21]:
# Capitalize 'city' in 'Richmond city'
print(regions.at[2944, 'CountyName'])
regions.at[2944, 'CountyName'] = 'Richmond City'
regions.at[2944, 'CountyName']

Richmond city


'Richmond City'

In [22]:
# Capitalize 'city' in 'St. Louis city'
print(regions.at[1597, 'CountyName'])
regions.at[1597, 'CountyName'] = 'St. Louis City'
regions.at[1597, 'CountyName']

St. Louis city


'St. Louis City'

In [23]:
# Split off 'county' from name
regions['CountyName'] = regions['CountyName'].apply(lambda x: x if 'County' not in x else ' '.join(x.split()[:-1]))

regions.head()

,State,CountyName,RegionName
0,Alabama,Autauga,Deep South
1,Alabama,Baldwin,Gulf Coast
2,Alabama,Barbour,Deep South
3,Alabama,Bibb,Appohzarka
4,Alabama,Blount,Appohzarka


In [24]:
# Combine 'county' and 'state' for merge
regions['county_state'] = regions['CountyName'] + regions.State

In [25]:
print("Regions shape:", regions.shape)
print("Birds shape:", us_birds.shape)

Regions shape: (3142, 4)
Birds shape: (1084942, 12)


In [26]:
merged = us_birds.merge(regions)
merged.shape

(1080701, 15)

In [27]:
# Bird rarity as a function of region and season

#table = pd.pivot_table(df, values='D', index=['A', 'B'],
                   # columns=['C'], aggfunc=np.sum)
season_region_ct = pd.pivot_table(merged, index='name', columns=['RegionName', 'season'], values='observ_count', aggfunc='count', fill_value=0.0)
season_region_ct

RegionName              Appohzarka                      Deep South         \
season                        Fall Spring Summer Winter       Fall Spring   
name                                                                        
Abert's Towhee                   0      0      0      0          0      0   
Acadian Flycatcher               7    108    165      0         19     82   
Acorn Woodpecker                 0      0      0      0          0      0   
African Silverbill               0      0      0      0          0      0   
Akekee                           0      0      0      0          0      0   
...                            ...    ...    ...    ...        ...    ...   
Yellow-throated Vireo           28    116     65      0         31    138   
Yellow-throated Warbler          9    109     47      1         32    149   
Yucatan Vireo                    0      0      0      0          0      0   
Zebra Dove                       0      0      0      0          0      0   
Zone-tailed Hawk                 0      0      0      0          0      0   

RegionName                            Great Lakes         ...  \
season                  Summer Winter        Fall Spring  ...   
name                                                      ...   
Abert's Towhee               0      0           0      0  ...   
Acadian Flycatcher          71      0           6     56  ...   
Acorn Woodpecker             0      0           0      0  ...   
African Silverbill           0      0           0      0  ...   
Akekee                       0      0           0      0  ...   
...                        ...    ...         ...    ...  ...   
Yellow-throated Vireo       57      0          19    122  ...   
Yellow-throated Warbler     48      9           0     22  ...   
Yucatan Vireo                0      0           0      0  ...   
Zebra Dove                   0      0           0      0  ...   
Zone-tailed Hawk             0      0           0      0  ...   

RegionName              Southern Florida        Southwest                \
season                            Summer Winter      Fall Spring Summer   
name                                                                      
Abert's Towhee                         0      0        36     82     51   
Acadian Flycatcher                     3      0         1     14      2   
Acorn Woodpecker                       0      0        51    143    123   
African Silverbill                     0      0         0      0      0   
Akekee                                 0      0         0      0      0   
...                                  ...    ...       ...    ...    ...   
Yellow-throated Vireo                  6      0         0     14      1   
Yellow-throated Warbler               20     56         1      8      0   
Yucatan Vireo                          0      0         0      0      0   
Zebra Dove                             0      0         0      0      0   
Zone-tailed Hawk                       0      0         3     38     35   

RegionName                     West Coast                       
season                  Winter       Fall Spring Summer Winter  
name                                                            
Abert's Towhee              69          2      7      6      3  
Acadian Flycatcher           0          0      0      0      0  
Acorn Woodpecker            30         79    120     89     86  
African Silverbill           0          4      3      5      3  
Akekee                       0          1      4      1      0  
...                        ...        ...    ...    ...    ...  
Yellow-throated Vireo        0          2      1      0      0  
Yellow-throated Warbler      6          0      2      0      0  
Yucatan Vireo                0          0      0      0      0  
Zebra Dove                   0         20     25     40     20  
Zone-tailed Hawk             1          3      0      1      2  

[923 rows x 56 columns]

In [28]:
# Bird rarity by region and season
def season_region_bird_rarity(bird, region, season):
    bird_percent = season_region_ct[(region, season)][bird] / season_region_ct[(region, season)].sum()
    if bird_percent > 0.005:
        return "Common"
    elif bird_percent > 0.001:
        return "Uncommon"
    else:
        return "Rare"
    
merged['bird_rareness'] = merged.apply(lambda x: season_region_bird_rarity(x['name'], x['RegionName'], x['season']), axis=1)
merged.bird_rareness.value_counts(normalize=True)

Common      0.633490
Uncommon    0.312713
Rare        0.053797
Name: bird_rareness, dtype: float64

In [29]:
merged.bird_rareness.value_counts(normalize=True)['Uncommon']

0.3127127669910549

In [30]:
merged[merged['bird_rareness'] == 'Rare']

,name,observ_count,country,state,county,latitude,longitude,observ_date,bad_name,month,season,county_state,State,CountyName,RegionName,bird_rareness
28,Eurasian Wigeon,1,United States,Illinois,Cook,41.707907,-87.886848,1939-04-30,1,4,Spring,CookIllinois,Illinois,Cook,Great Lakes,Rare
78,Yellow-bellied Flycatcher,2,United States,Illinois,Cook,41.850030,-87.650050,1972-05-27,1,5,Spring,CookIllinois,Illinois,Cook,Great Lakes,Rare
80,Wilson's Phalarope,2,United States,Illinois,Cook,41.850030,-87.650050,1972-05-27,1,5,Spring,CookIllinois,Illinois,Cook,Great Lakes,Rare
88,Vermilion Flycatcher,1,United States,Illinois,Cook,41.850030,-87.650050,1986-04-16,1,4,Spring,CookIllinois,Illinois,Cook,Great Lakes,Rare
91,American Bittern,1,United States,Illinois,Cook,41.630592,-87.865498,1947-10-19,1,10,Fall,CookIllinois,Illinois,Cook,Great Lakes,Rare
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1080678,Lark Bunting,1,United States,South Dakota,Hand,44.525687,-98.988490,1977-06-15,1,6,Summer,HandSouth Dakota,South Dakota,Hand,Heartland,Rare
1080691,Eurasian Collared-Dove,10,United States,Illinois,Montgomery,39.301054,-89.285502,1998-06-22,1,6,Summer,MontgomeryIllinois,Illinois,Montgomery,Heartland,Rare
1080693,Swainson's Warbler,6,United States,West Virginia,McDowell,37.417891,-81.875782,1997-06-04,1,6,Summer,McDowellWest Virginia,West Virginia,McDowell,Appohzarka,Rare
1080694,Mountain Bluebird,1,United States,Missouri,Daviess,39.974710,-93.801020,1993-01-02,1,1,Winter,DaviessMissouri,Missouri,Daviess,Heartland,Rare


In [31]:
import category_encoders as ce
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

In [33]:
merged['target'] = merged['bird_rareness'].map({"Common": 0,
                                        "Uncommon": 1,
                                        "Rare": 2})

merged.head()

,name,observ_count,country,state,county,latitude,longitude,observ_date,bad_name,month,season,county_state,State,CountyName,RegionName,bird_rareness,target
0,Magnolia Warbler,2,United States,Illinois,Cook,41.775629,-87.583273,1995-08-27,1,8,Summer,CookIllinois,Illinois,Cook,Great Lakes,Uncommon,1
1,Great Blue Heron,2,United States,Illinois,Cook,41.956746,-87.913027,1999-07-24,1,7,Summer,CookIllinois,Illinois,Cook,Great Lakes,Common,0
2,Purple Martin,6,United States,Illinois,Cook,41.707907,-87.886848,1965-06-19,1,6,Summer,CookIllinois,Illinois,Cook,Great Lakes,Common,0
3,Common Yellowthroat,3,United States,Illinois,Cook,41.707907,-87.886848,1965-06-19,1,6,Summer,CookIllinois,Illinois,Cook,Great Lakes,Common,0
4,Eastern Towhee,3,United States,Illinois,Cook,41.707907,-87.886848,1965-06-19,1,6,Summer,CookIllinois,Illinois,Cook,Great Lakes,Uncommon,1


In [34]:
merged.target.value_counts(normalize=True)

0    0.633490
1    0.312713
2    0.053797
Name: target, dtype: float64

In [38]:
features = ['name', 'season', 'RegionName']
target = 'target'

X = merged[features]
y = merged[target]

encoder = ce.CatBoostEncoder()

X = encoder.fit_transform(X, y)

X

,name,season,RegionName
0,0.420308,0.420308,0.420308
1,0.420308,0.710154,0.710154
2,0.420308,0.473436,0.473436
3,0.420308,0.355077,0.355077
4,0.420308,0.284062,0.284062
...,...,...,...
1080696,0.135564,0.447222,0.371764
1080697,0.029583,0.324714,0.356172
1080698,1.058692,0.410737,0.371755
1080699,0.208771,0.410743,0.355593


In [39]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

X_train.shape, X_test.shape, y_train.shape, y_test.shape

((864560, 3), (216141, 3), (864560,), (216141,))

In [ ]:
forest = RandomForestClassifier(n_estimators=1000, max_depth=None, random_state=42)

forest.fit(X_train, y_train)

forest.score(X_test, y_test)

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

